# Calling the LLM 🧑‍💻

In this notebook, we'll get familiar with calling the LLM API from code.

In addition, we'll extend upon regular prompting by exploring how we can make the LLM generate structured outputs.

Before we start, it's important to understand our package structure. As a warm up exercise, follow the instructions below to get familiar with the package structure.

> **Exercise** 📝
>
> - Inspect the `pyproject.toml`. This defines the project dependencies and configuration.
> - Inspect the `src` folder. This is where the source code, or our package, lives.
> - Navigate to the `generation` module in our package. What functions does it define?

As you see, even though all exercises are in notebooks (for storytelling and visualization purposes), all of our important code lives in our package. 

We will use and extend our package throughout this training. Structuring your code like this is important to allow for better reusability, testability and maintainability.

## Regular prompting

> **Exercise** 📝
>
> - Before we call the LLM from code, take a minute to get familiar with the [Azure OpenAI playground in the portal](https://portal.azure.com/#view/Microsoft_Azure_ProjectOxford/CognitiveServicesHub/~/OpenAI) if you haven't already.
> - Check out the [API reference](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference) to see how we can call the LLM from code.
> - Now, fill in the TODO's in the `generation.py` module in `./src/llmops_training`, so we can call the LLM from code!

In [1]:
from llmops_training.news_reader.generation import generate_text

In [2]:
prompt = "Hi!"
generate_text(prompt)

'Hi there! How can I help you today?'

## Extracting structured information (naive)

A common use case is to extract structured information from text. 

For example, given the text "XYZ's stock price is expected to increase", we might want to extract:
- business: "XYZ"
- stock price change: "increase"

and store it in a data base.

One way to do so, is to *ask* the LLM to generate the structured information for us in JSON format.

> **Exercise** 📝
>
> - Fill in the TODO below to try to extract structured information form the given sentence.
> - Check if the output is in correct JSON format

In [3]:
from llmops_training.news_reader.utils import check_json

In [4]:
sentence = "XYZ's stock price is expected to increase"

prompt_template = """
Extract the following fields in JSON format:
   - "business": "The business involved in the text (str)",
   - "stock_price_change": "Whether stock price will 'increase' or 'decrease' (str)"
From the following sentence:
{sentence}
"""

prompt = prompt_template.format(sentence=sentence)
output = generate_text(prompt)
print(output)

#output = "Not implemented yet!" # TODO: Fill me in! Call the LLM

{
  "business": "XYZ",
  "stock_price_change": "increase"
}


In [ ]:
# %load ../solutions/calling-the-llm/generate-text.py

Now let's see if that worked.

In [5]:
check_json(output)
print(output)

Valid JSON! 🎉
{
  "business": "XYZ",
  "stock_price_change": "increase"
}


^ does that return "_Not a valid JSON!_"? That's okay! We expected that.

As you see, the LLM is free to format the output however it wants. It's not guaranteed to output a correct JSON structure. 

It gets close though, and with a few iterations we can probably get what we want, but there's a better way.

## Extracting structured information (instructor)

One way to guarantee the output is in the correct format, is to use [function calling](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling).

Here, we pass a structured definition of the desired outputs to the LLM, and it will return a structured object.

Now, there is a tool, called [Instructor](https://python.useinstructor.com/) that builds on this capability, and allows you to define a Pydantic schema for the LLM to generate. Let's see how that works.

In [7]:
from pydantic import BaseModel, Field
from typing import Literal

from llmops_training.news_reader.generation import generate_object

Here we define the schema:

In [8]:
class StructuredInfo(BaseModel):
    business: str = Field(..., description="Business involved in the text")
    stock_price_change: Literal["increase", "decrease"] = Field(..., description="Likely stock price change")
    
prompt_template = "Extract structured info from the sentence: {sentence}"

prompt = prompt_template.format(sentence=sentence)

In [9]:
output = generate_object(prompt, StructuredInfo)
output

StructuredInfo(business='XYZ', stock_price_change='increase')

With `instructor` it's guaranteed in the right format!

Now let's put this to use in our use case.

---